# 🖊️ SignVerifAI - Critical Optimization v5.0

## Key Fixes Applied:
- 🎯 **80% Skilled Forgery Negatives** (was 50%)
- 🚫 **No Blur/ColorJitter** (destroys stroke details)
- 📐 **Embedding 256** (was 128)
- 📊 **Contrastive Loss** (more stable than ArcFace)
- ⚡ **Higher LR** (backbone 3e-5, head 5e-4)
- 🧊 **5 Freeze Epochs** (proper warmup)

**Expected:** AUC >= 0.94, EER <= 0.10

---

In [ ]:
# 1. Setup
!nvidia-smi
!rm -rf SignVerifAI
!git clone https://github.com/gorkemelih/SignVerifAI.git
%cd SignVerifAI
!pip install -e . -q
print("✅ Kurulum tamamlandı!")

In [ ]:
# 2. Mount Drive & Load Data
from google.colab import drive
drive.mount('/content/drive')

import os
!unzip -q -o /content/drive/MyDrive/SignVerifAI/data_processed.zip -d .
print("✅ Veriler yüklendi!")

In [ ]:
# 3. Prepare Data (80% skilled forgery negatives!)
!signverify split
!signverify pairs

## 🚀 Training Experiments

### EXP-1: Optimized Contrastive (RECOMMENDED)

In [ ]:
# EXP-1: MobileNet + Contrastive + 80% skilled forgery
!signverify train \
    --device cuda \
    --epochs 50 \
    --batch-size 128 \
    --backbone mobilenet_v3_large \
    --embedding-dim 256 \
    --freeze-epochs 5 \
    --backbone-lr 3e-5 \
    --head-lr 5e-4 \
    --loss contrastive \
    --hard-mining \
    --scheduler cosine \
    --log-every 5

### EXP-2: Triplet Loss + Batch-Hard

In [ ]:
# EXP-2: Triplet + Batch-Hard (alternative)
# !signverify train \
#     --device cuda \
#     --epochs 50 \
#     --backbone efficientnet_b0 \
#     --embedding-dim 256 \
#     --loss triplet \
#     --batch-hard \
#     --scheduler cosine

In [ ]:
# 4. Evaluate
!signverify eval --device cuda --threshold-mode all

In [ ]:
# 5. View Results
from IPython.display import Image, display, Markdown

display(Image('outputs/reports/roc_curve.png'))
display(Image('outputs/reports/score_distribution.png'))

with open('outputs/reports/eval_report.md', 'r') as f:
    display(Markdown(f.read()))

In [ ]:
# 6. Training History
import pandas as pd
import matplotlib.pyplot as plt

history = pd.read_csv('outputs/reports/train_history.csv')

fig, axes = plt.subplots(2, 2, figsize=(12, 8))
axes[0,0].plot(history['epoch'], history['train_loss'], label='Train')
axes[0,0].plot(history['epoch'], history['val_loss'], label='Val')
axes[0,0].set_title('Loss'); axes[0,0].legend()

axes[0,1].plot(history['epoch'], history['auc'])
axes[0,1].axhline(y=0.94, color='r', linestyle='--', label='Target 0.94')
axes[0,1].set_title('AUC'); axes[0,1].legend()

axes[1,0].plot(history['epoch'], history['eer'])
axes[1,0].axhline(y=0.10, color='r', linestyle='--', label='Target 0.10')
axes[1,0].set_title('EER (lower is better)'); axes[1,0].legend()

axes[1,1].plot(history['epoch'], history['lr'])
axes[1,1].set_title('Learning Rate')

plt.tight_layout()
plt.show()

In [ ]:
# 7. Save to Drive
import shutil, glob

runs = sorted(glob.glob('outputs/models/run_*'))
if runs:
    dest = "/content/drive/MyDrive/SignVerifAI/trained_models/"
    os.makedirs(dest, exist_ok=True)
    for f in glob.glob(f"{runs[-1]}/*"):
        shutil.copy(f, dest)
    for f in glob.glob('outputs/reports/*'):
        shutil.copy(f, "/content/drive/MyDrive/SignVerifAI/reports/")
    print("✅ Saved!")

---
## 📋 v5.0 Optimization Summary

| Change | Before | After |
|--------|--------|-------|
| Negative Sampling | 50% skilled | **80% skilled** |
| ColorJitter/Blur | ✅ | **❌ Removed** |
| Embedding Dim | 128 | **256** |
| Loss | ArcFace | **Contrastive** |
| Backbone LR | 1e-5 | **3e-5** |
| Head LR | 3e-4 | **5e-4** |
| Freeze Epochs | 3 | **5** |
| Scheduler | OneCycle | **Cosine** |